In [ ]:
import os

from aavomics import database
from aavomics import aavomics
import scipy
import anndata
import pandas
import scanpy
import numpy

import plotly.graph_objects as graph_objects
from plotly import offline as plotly
scanpy.settings.figdir = "out"

In [ ]:
ANNDATA_FILE_NAME = "aavomics_mouse_cortex_2021.h5ad"
TAXONOMY_NAME = "CCN202105051"

In [ ]:
adata = anndata.read_h5ad(os.path.join(database.DATA_PATH, ANNDATA_FILE_NAME))

In [ ]:
non_neuron_adata = adata[~adata.obs[TAXONOMY_NAME].isin(["nan", "Unknown", "Multiplets"])].copy()

In [ ]:
transcript_counts_sum = numpy.mean(non_neuron_adata.X.sum(axis=1))

In [ ]:
scanpy.pp.normalize_total(non_neuron_adata, target_sum=transcript_counts_sum, inplace=True)

In [ ]:
cell_type_marker_genes = {
    "Astrocytes": [
        "Sox9",
        "Myoc",
        "Gfap",
        "Fxyd6"
    ],
    "Vascular Cells": [
        "Cldn5",
        "Ly6a",
        "Ly6c1",
        "Pdgfrb",
        "Abcc9",
    ],
    "OPCs": [
        "Pdgfra",
        "Cspg4",
        "Pbk",
        "Spc24",
        "Smc2",
        "Lmnb1"
    ]
}

cell_type_marker_ensembl_ids = {}

for cell_type in cell_type_marker_genes:
    cell_type_marker_ensembl_ids[cell_type] = []
    
    for gene_name in cell_type_marker_genes[cell_type]:
        ensembl_id = adata.var[adata.var["Gene Name"] == gene_name].index.values[0]
        cell_type_marker_ensembl_ids[cell_type].append(ensembl_id)

In [ ]:
cell_type_order = [
    "Myoc- Astrocytes",
    "Myoc+ Astrocytes",
    "Endothelial Cells",
    "Pericytes",
    "Red Blood Cells",
    "Vascular SMCs",
    "VLMCs",
    "Microglia",
    "Perivascular Macrophages",
    "Leukocytes",
    "OPCs",
    "Committed Oligodendrocytes",
    "Mature Oligodendrocytes"
]

scanpy.pl.dotplot(
    non_neuron_adata,
    cell_type_marker_genes,
    categories_order=cell_type_order,
    groupby=TAXONOMY_NAME,
    dendrogram=False,
    gene_symbols="Gene Name",
    log=True,
    save="non_neuron_subtypes.svg"
)